In [1]:
import pandas as pd

In [2]:
# Load datasets
status_analysis = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Status_Analysis.csv")
service_options = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Service_Options.csv")
online_services = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Online_Services.csv")
payment_info = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Payment_Info.csv")
location_data = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Location_Data.csv")
customer_info = pd.read_csv("C:/Users/Owner/Desktop/archive (1)/Status_Analysis.csv")

In [3]:
# Merge datasets on customer_id
df = status_analysis.merge(service_options, on='customer_id') # 만족도 점수, 이탈 라벨, 이탈률 점수 + 서비스 이용 패턴(친구 추천, 여러 전화선 가입 여부 등)
df = df.merge(online_services, on='customer_id') #  온라인 서비스 가입 여부
df = df.merge(payment_info, on='customer_id') # 결제 방법, 월간 요금, 총 요금

In [8]:
df = df.drop(columns=['churn_label']) 
df = df.drop(columns=['churn_score'])
# Churn Value -> 목표 변수수

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   customer_id                        7043 non-null   object 
 1   satisfaction_score                 7043 non-null   int64  
 2   cltv                               7043 non-null   int64  
 3   customer_status                    7043 non-null   object 
 4   churn_value                        7043 non-null   int64  
 5   churn_category                     7043 non-null   object 
 6   churn_reason                       1869 non-null   object 
 7   tenure                             7043 non-null   int64  
 8   internet_service_x                 7043 non-null   object 
 9   phone_service_x                    7043 non-null   object 
 10  multiple_lines                     7043 non-null   object 
 11  avg_monthly_gb_download            7043 non-null   int64

In [10]:
print(df.describe()) # 기술 통계

       satisfaction_score         cltv  churn_value       tenure  \
count         7043.000000  7043.000000  7043.000000  7043.000000   
mean             3.244924  4400.295755     0.265370    32.371149   
std              1.201657  1183.057152     0.441561    24.559481   
min              1.000000  2003.000000     0.000000     0.000000   
25%              3.000000  3469.000000     0.000000     9.000000   
50%              3.000000  4527.000000     0.000000    29.000000   
75%              4.000000  5380.500000     1.000000    55.000000   
max              5.000000  6500.000000     1.000000    72.000000   

       avg_monthly_gb_download  number_of_referrals  monthly_ charges  \
count              7043.000000          7043.000000       7043.000000   
mean                 20.515405             1.951867         64.761692   
std                  20.418940             3.001199         30.090047   
min                   0.000000             0.000000         18.250000   
25%                   

In [11]:
# 결측값이 있는 컬럼 확인
missing_columns = df.columns[df.isnull().any()]

In [13]:
# 결측값이 있는 컬럼과 해당 컬럼의 결측값 개수 출력
print(df[missing_columns].isnull().sum())

churn_reason     5174
offer            3877
internet_type    1526
dtype: int64


In [15]:
# 카테고리형 컬럼 확인
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

print("카테고리형 컬럼:")
print(categorical_columns)

카테고리형 컬럼:
Index(['customer_id', 'customer_status', 'churn_category', 'churn_reason',
       'internet_service_x', 'phone_service_x', 'multiple_lines',
       'unlimited_data', 'offer', 'referred_a_friend', 'phone_service_y',
       'internet_service_y', 'online_security', 'online_backup',
       'device_protection', 'premium_tech_support', 'streaming_tv',
       'streaming_movies', 'streaming_music', 'internet_type', 'contract',
       'paperless_billing', 'payment_method'],
      dtype='object')


In [17]:
# 카테고리형 변수 확인
categorical_columns = ['customer_id', 'customer_status', 'churn_category', 'churn_reason',
       'internet_service_x', 'phone_service_x', 'multiple_lines',
       'unlimited_data', 'offer', 'referred_a_friend', 'phone_service_y',
       'internet_service_y', 'online_security', 'online_backup',
       'device_protection', 'premium_tech_support', 'streaming_tv',
       'streaming_movies', 'streaming_music', 'internet_type', 'contract',
       'paperless_billing', 'payment_method']
print("\n카테고리형 변수의 고유값:")
for col in categorical_columns:
    print(f"{col}: {df[col].unique()}")


카테고리형 변수의 고유값:
customer_id: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
customer_status: ['Stayed' 'Churned' 'Joined']
churn_category: ['Not Applicable' 'Competitor' 'Dissatisfaction' 'Other' 'Price'
 'Attitude']
churn_reason: [nan 'Competitor had better devices' 'Product dissatisfaction'
 'Network reliability' 'Limited range of services'
 'Competitor made better offer' "Don't know" 'Long distance charges'
 'Attitude of service provider' 'Attitude of support person'
 'Competitor offered higher download speeds'
 'Competitor offered more data' 'Lack of affordable download/upload speed'
 'Deceased' 'Moved' 'Service dissatisfaction' 'Price too high'
 'Lack of self-service on Website' 'Poor expertise of online support'
 'Extra data charges' 'Poor expertise of phone support']
internet_service_x: ['Yes' 'No']
phone_service_x: ['Yes' 'No']
multiple_lines: ['No' 'Yes']
unlimited_data: ['Yes' 'No']
offer: [nan 'Offer E' 'Offer D' 'Offer A' 'Offer B' 'Off

In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

# 카테고리형 변수 리스트 (customer_id는 제외, 고유 식별자로 분석에 적합하지 않음)
categorical_columns = [
    'customer_status', 'churn_category', 'churn_reason', 'internet_service_x',
    'phone_service_x', 'multiple_lines', 'unlimited_data', 'offer',
    'referred_a_friend', 'phone_service_y', 'internet_service_y',
    'online_security', 'contract', 'paperless_billing', 'payment_method'
]

# 결측값 처리 (NaN을 'Unknown'으로 대체)
df[categorical_columns] = df[categorical_columns].fillna('Unknown')

# Label Encoding
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 카이제곱 검정을 수행하는 함수
def chi_square_test(data, category_columns, target_column):
    results = {}
    for col in category_columns:
        # 교차표 생성
        contingency_table = pd.crosstab(data[col], data[target_column])
        
        # 카이제곱 검정 수행
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        # 결과 저장
        results[col] = {
            'Chi2 Statistic': chi2,
            'p-value': p,
            'Degrees of Freedom': dof
        }
        
    return results

# Churn Value(목표 변수)가 포함된 데이터 확인
target_column = 'churn_value'

# 카이제곱 검정 실행
chi2_results = chi_square_test(df, categorical_columns, target_column)

# 결과 출력
for col, result in chi2_results.items():
    print(f"Feature: {col}")
    print(f"  Chi2 Statistic: {result['Chi2 Statistic']}")
    print(f"  p-value: {result['p-value']}")
    print(f"  Degrees of Freedom: {result['Degrees of Freedom']}")
    print("-" * 40)

# 인코딩된 고유값 확인 (역변환 가능)
for col, le in label_encoders.items():
    print(f"Unique values for {col}: {list(le.classes_)}")


Feature: customer_status
  Chi2 Statistic: 7043.0
  p-value: 0.0
  Degrees of Freedom: 2
----------------------------------------
Feature: churn_category
  Chi2 Statistic: 7042.999999999999
  p-value: 0.0
  Degrees of Freedom: 5
----------------------------------------
Feature: churn_reason
  Chi2 Statistic: 7043.0
  p-value: 0.0
  Degrees of Freedom: 20
----------------------------------------
Feature: internet_service_x
  Chi2 Statistic: 364.51979899428943
  p-value: 2.9203003727695145e-81
  Degrees of Freedom: 1
----------------------------------------
Feature: phone_service_x
  Chi2 Statistic: 0.9150329892546948
  p-value: 0.3387825358066928
  Degrees of Freedom: 1
----------------------------------------
Feature: multiple_lines
  Chi2 Statistic: 11.143251001456251
  p-value: 0.0008433795342472428
  Degrees of Freedom: 1
----------------------------------------
Feature: unlimited_data
  Chi2 Statistic: 194.5492167239403
  p-value: 3.2314344412572294e-44
  Degrees of Freedom: 1
----